<a href="https://colab.research.google.com/github/steven4354/jsonformer-test/blob/main/jsonformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install jsonformer

from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set the folder path in Google Drive for model files
google_drive_model_path = "/content/drive/My Drive/Colab Notebooks/dolly-v2-12b"

# Check if the model folder exists
if not os.path.exists(google_drive_model_path):
    # Create folder in Google Drive
    os.makedirs(google_drive_model_path)
    
    # Download model and tokenizer
    model_name = "databricks/dolly-v2-12b"
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Save model and tokenizer to Google Drive
    model.save_pretrained(google_drive_model_path)
    tokenizer.save_pretrained(google_drive_model_path)

# Load model and tokenizer from Google Drive
model = AutoModelForCausalLM.from_pretrained(google_drive_model_path)
tokenizer = AutoTokenizer.from_pretrained(google_drive_model_path)

# Use the loaded model and tokenizer with Jsonformer
from jsonformer import Jsonformer

json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "items": {"type": "string"}
        }
    }
}

prompt = "Generate a person's information based on the following schema:"
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt)
generated_data = jsonformer()

print(generated_data)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

{'name': 'age', 'age': 1.000000000000001e+18, 'is_student': False, 'courses': ['Java', 'Python']}


In [3]:
import time
from jsonformer import Jsonformer

json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "items": {"type": "string"}
        }
    }
}

prompt = "Generate a person's information based on the following schema:"
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt)

start_time = time.time()
generated_data = jsonformer()
end_time = time.time()

elapsed_time = end_time - start_time
print("Generated data:", generated_data)
print("Time taken to generate the data: {:.2f} seconds".format(elapsed_time))

Generated data: {'name': 'age', 'age': 1.000000000000001e+18, 'is_student': False, 'courses': ['Java', 'Python']}
Time taken to generate the data: 77.19 seconds


In [4]:
"""
# TODO(@steven4354): debug this later
# This section of code doesn't work on colab pro+, crashes and says it ran out of memory
# oddly not GPU memory was used and the max system RAM when to ~50GB (out of 83.5 GB available)


# Install necessary packages
!pip install transformers
!pip install jsonschema
!pip install jsonformer

# Import necessary libraries
import threading
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from jsonformer import Jsonformer as OriginalJsonformer

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to monitor GPU memory usage
def get_gpu_memory_usage():
    return torch.cuda.memory_allocated() / (1024 ** 2)

# Function to monitor and print memory usage at regular intervals
def print_memory_usage(interval=1):
    while monitor_memory:
        print(f"GPU memory usage: {get_gpu_memory_usage()} MB")
        time.sleep(interval)

# Custom Jsonformer class with memory usage monitoring
class Jsonformer(OriginalJsonformer):
    def _generate_text(self, input_ids):
        global monitor_memory
        monitor_memory = True
        memory_monitor_thread = threading.Thread(target=print_memory_usage)
        memory_monitor_thread.start()
        generated_text = super()._generate_text(input_ids)
        monitor_memory = False
        memory_monitor_thread.join()
        return generated_text

# Load model and tokenizer from Google Drive
model = AutoModelForCausalLM.from_pretrained(google_drive_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(google_drive_model_path)

# Use the loaded model and tokenizer with the custom Jsonformer
json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "items": {"type": "string"}
        }
    }
}

prompt = "Generate a person's information based on the following schema:"
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt)
generated_data = jsonformer()

print(generated_data)
"""

'\n# TODO(@steven4354): debug this later\n# This section of code doesn\'t work on colab pro+, crashes and says it ran out of memory\n# oddly not GPU memory was used and the max system RAM when to ~50GB (out of 83.5 GB available)\n\n\n# Install necessary packages\n!pip install transformers\n!pip install jsonschema\n!pip install jsonformer\n\n# Import necessary libraries\nimport threading\nimport time\nimport torch\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\nfrom jsonformer import Jsonformer as OriginalJsonformer\n\n# Set the device to GPU if available\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n\n# Function to monitor GPU memory usage\ndef get_gpu_memory_usage():\n    return torch.cuda.memory_allocated() / (1024 ** 2)\n\n# Function to monitor and print memory usage at regular intervals\ndef print_memory_usage(interval=1):\n    while monitor_memory:\n        print(f"GPU memory usage: {get_gpu_memory_usage()} MB")\n        time.sleep(interval